# Thread Data and Race Conditions

We saw at the end of the last lesson that threads can access to share data. This is useful to share configuration, but it introduces problems. The main problem that arises is _[**race conditions**](https://en.wikipedia.org/wiki/Race_condition)_. Let us create an example.

In [1]:
from threading import Thread, current_thread, Lock
from time import sleep, time
from sys import stderr

## Shared state

Previously we had created a dictionary and had issues arrive when different threads update values.  But actually, one simple share scalar value suffices to see the problem.

In [2]:
def increment(n):
    global counter
    for _ in range(n):
        counter += 1

In [6]:
counter, nthread, nloop = 0, 100, 55_000 
threads = [Thread(target=increment, args=(nloop,)) for _ in range(nthread)]
for t in threads:
    t.start()

# Make sure they have finished before we report
while alive := sum(t.is_alive() for t in threads):
    sleep(2)
    print("Num threads alive:", sum(t.is_alive() for t in threads))
    
print(f"{nloop:,} loops X {nthread:,} threads -> counter is {counter:,}")    

Num threads alive: 0
55,000 loops X 100 threads -> counter is 4,872,301


### Where we went wrong?

A subtlety in our program is that the single augmented assignment, `+=` is not **atomic**. 

Behind the scenes, `+=` runs 2 (or more) operations. In pseudo-code, this looks like:

```
add(counter, 1) TO <temp>  # A
store("counter", <temp>)   # B
```

Multiple threads are executing concurrently. Hence one or more may hold a value that becomes outdated between the time when the addition is performed (A) and the time when the name `counter` is rebound (B).

To be more exact, we can actually look at the Python bytecode (wordcode).  A thread may be suspended after any single instruction, and there are **four** instructions involved in the single augmented assignment.

In [7]:
import dis
dis.dis(increment)

  3           0 LOAD_GLOBAL              0 (range)
              2 LOAD_FAST                0 (n)
              4 CALL_FUNCTION            1
              6 GET_ITER
        >>    8 FOR_ITER                12 (to 22)
             10 STORE_FAST               1 (_)

  4          12 LOAD_GLOBAL              1 (counter)
             14 LOAD_CONST               1 (1)
             16 INPLACE_ADD
             18 STORE_GLOBAL             1 (counter)
             20 JUMP_ABSOLUTE            8
        >>   22 LOAD_CONST               0 (None)
             24 RETURN_VALUE


## Thread Synchronization

<img src="recording_studio_light.png" width="25%" align="right"/>How can we fix the race condition? 

We need a way to keep the threads from stepping onto each other's data, some signal that a resource is **busy**.

*(Example of INEs studios, a recording light is on, the studio is busy, nobody will enter the room)*

<img src="recording_studio_light.png" width="25%" align="right"/>The most basic synchronization mechanism is a [Lock](https://en.wikipedia.org/wiki/Lock_(computer_science)), or a Mutex (mutual exclusion lock). Python includes the very intuitive `threading.Lock` class. 

A Lock works like the RECORDING light pictured. The first person (thread) that "arrives" turns on the light (acquires the lock). Anyone else has to wait for the person/thread to turn the light off and make the room (resource) available again.

## Locking

In [8]:
lock = Lock()

def lock_hogger(lock, wait=5):
    name = current_thread().name
    print(f"{name}: acquiring lock.")
    lock.acquire()
    print(f"{name}: Lock acquired, sleeping")
    sleep(wait)
    print(f"{name}: Woke up, releasing lock")
    lock.release()

Thread(target=lock_hogger, args=(lock,)).start()

Thread-404: acquiring lock.
Thread-404: Lock acquired, sleeping
Thread-404: Woke up, releasing lock


We can ask about the state of a lock.

In [9]:
print("Is lock currently being used?", lock.locked())

# Claim the lock
lock.acquire()
print("Lock acquired?", lock.locked())

# We are done now
lock.release()
print("Still being used?", lock.locked())

Is lock currently being used? False
Lock acquired? True
Still being used? False


If we only ever ran sequential code, the lock would just be equivalent to a Boolean value.  But it is shared across all threads that have it in scope.  When another thread tries to acquire a lock that is in use, it will block until the lock becomes free.

It is important (and sometimes tricky) to get the sequence of actions right, since a released lock will raise an exception if you try to release it again.

In [10]:
lock = Lock()
t = Thread(target=lock_hogger, args=(lock, 0.1))
t.start()
print("Releasing lock in MainThread...")
lock.release()

Thread-405: acquiring lock.
Thread-405: Lock acquired, sleeping
Releasing lock in MainThread...


Exception in thread Thread-405:
Traceback (most recent call last):
  File "/home/dmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/dmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-3315af7a6c72>", line 10, in lock_hogger
RuntimeError: release unlocked lock


Thread-405: Woke up, releasing lock


Where we get an exception among the competing `lock.release()` calls depends on the timings of threads.

In [11]:
lock = Lock()
t = Thread(target=lock_hogger, args=(lock, 0))
t.start()
sleep(0.1)
try:
    print("Releasing lock in MainThread...")
    lock.release()
except Exception as err:
    print(f"{repr(err)} in {current_thread().name}", file=stderr)

Thread-406: acquiring lock.
Thread-406: Lock acquired, sleeping
Thread-406: Woke up, releasing lock
Releasing lock in MainThread...


RuntimeError('release unlocked lock') in MainThread


## Fixing the counter

Now that we know about locks, we can use them to fix our counter example:

In [12]:
lock, counter, nthread, nloop = Lock(), 0, 100, 70_000 

def increment(n, lock):
    global counter
    for _ in range(n):
        lock.acquire()
        counter += 1
        lock.release()

threads, now = [], time()
for _ in range(nthread):
    t = Thread(target=increment, args=(nloop, lock))
    threads.append(t)
    t.start()

In [13]:
while alive := sum(t.is_alive() for t in threads):
    sleep(5)
    print("Num threads alive:", sum(t.is_alive() for t in threads))
    
print(f"{nloop:,} loops X {nthread:,} threads -> counter is {counter:,}")
print(f"Running everything took {time()-now:.2f} seconds!")

Num threads alive: 98
Num threads alive: 98
Num threads alive: 98
Num threads alive: 98
Num threads alive: 98
Num threads alive: 98
Num threads alive: 98
Num threads alive: 98
Num threads alive: 97
Num threads alive: 0
70,000 loops X 100 threads -> counter is 7,000,000
Running everything took 60.98 seconds!


## Problems with synchronization

Locks are acquired before accessing what we call "Critical Sections"; important sections in our code that can potentially introduce race conditions. 

The problem is that locks are "cooperative", but you're not obliged to use them. If just one function in the codebase does lock management wrong, the problems can propagate. 

Possible problems:

1. You fail to recognize that there is a "critical section".
2. You fail to acquire the lock before entering the critical section.
3. The critical section might use resources NOT protected by the lock.
4. You fail to release the lock (e. g. code breaks before releasing).
5. Deadlocks! (more in next lesson).

### Exceptions

What if our function doesn't run has hoped?

In [14]:
lock = Lock()

def faulty_lock_handler(lock, wait=10):
    print("\t\tThread: Acquiring lock.")
    lock.acquire()
    print("\t\tThread: Lock acquired")
    sleep(wait)
    print("\t\tThread: Woke up, releasing lock")
    lock.release()

In [15]:
# The `wait` param is incorrect, should be a number
Thread(target=faulty_lock_handler, args=(lock, 'x')).start()

		Thread: Acquiring lock.
		Thread: Lock acquired


Exception in thread Thread-507:
Traceback (most recent call last):
  File "/home/dmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/dmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-14-1e7a59c79ed2>", line 7, in faulty_lock_handler
TypeError: an integer is required (got type str)


Trying to acquire the lock will block **FOREVER**:

In [16]:
lock.acquire()

KeyboardInterrupt: 

We could potentially add a _timeout_ to our acquire method, it will block for the specified number of seconds; if it hasn't acquired the lock, it returns `False`:

In [17]:
lock.acquire(timeout=2)

False

We can make it non-blocking, if it is not able to acquire the lock, it will release it immediately:

In [18]:
lock.acquire(blocking=False)

False

We have a handle on the lock variable, so we can release.  This is dangerous though: what if the thread really **is** still using the resource?

In [19]:
lock.release()
lock.acquire(blocking=False)

True

## Context Manager

To greatly ease this problem, we can use locks as Context Managers. This will release the lock **even if** something goes wrong within the critical section:

In [20]:
def fixed_lock_handler(lock, wait=10):
    print("\t\tThread: Acquiring lock.")
    with lock:
        print("\t\tThread: Lock acquired")
        sleep(wait)
    print("\t\tThread: Woke up, releasing lock")

In [21]:
lock = Lock()
Thread(target=fixed_lock_handler, args=(lock, 5)).start()

		Thread: Acquiring lock.
		Thread: Lock acquired
		Thread: Woke up, releasing lock


In [22]:
lock.acquire()

True

In [23]:
lock.release()

In [24]:
# The `wait` param is incorrect, should be a number
Thread(target=fixed_lock_handler, args=(lock, 'x')).start()

		Thread: Acquiring lock.
		Thread: Lock acquired


Exception in thread Thread-509:
Traceback (most recent call last):
  File "/home/dmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/dmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-20-8d28d3b54873>", line 5, in fixed_lock_handler
TypeError: an integer is required (got type str)


Is the lock still acquired?

In [25]:
lock.locked()

False

In [26]:
lock.acquire()

True

In [27]:
lock.release()

The critical section failed with an exception, but the lock was released before exiting. Using `with` and the lock as context manager is syntactic sugar for this pattern:

```python
lock.acquire()
try:
    critical_section()
finally:
    lock.release()  # We'll release the lock no matter what
```

## Improving the threaded counter

The last touch for our counter should be to use the context manager protocol of the lock object:

In [28]:
lock, counter, nthread, nloop = Lock(), 0, 100, 50_000 

def increment(n, lock):
    global counter
    for _ in range(n):
        with lock:
            counter += 1
            
threads, now = [], time()
for _ in range(nthread):
    t = Thread(target=increment, args=(nloop, lock))
    threads.append(t)
    t.start()

In [29]:
while alive := sum(t.is_alive() for t in threads):
    sleep(5)
    print("Num threads alive:", sum(t.is_alive() for t in threads))
    
print(f"{nloop:,} loops X {nthread:,} threads -> counter is {counter:,}")
print(f"Running everything took {time()-now:.2f} seconds!")

Num threads alive: 100
Num threads alive: 100
Num threads alive: 100
Num threads alive: 100
Num threads alive: 100
Num threads alive: 99
Num threads alive: 99
Num threads alive: 0
50,000 loops X 100 threads -> counter is 5,000,000
Running everything took 43.86 seconds!


## Coarser Locks

The threaded program is a **whole lot** slower than a serial version! When we do the augmented assignment, `+=` it is not *atomic*, but it is still a lot less work than acquiring and releasing the lock.  The program is dominated by juggling locks.

In [30]:
%%time
counter, nthread, nloop = 0, 100, 50_000 

def increment(nthread, nloop):
    global counter
    for n in range(nloop):
        for m in range(nthread):
            counter += 1
            
increment(nthread, nloop)

CPU times: user 567 ms, sys: 0 ns, total: 567 ms
Wall time: 568 ms


Even the threaded version that did not use locks will be slower than the serial version, but not hundreds of times slower.  By releasing locks less often, and doing more work within a thread, we can make a compromise.

In [31]:
lock, counter, nthread, nloop = Lock(), 0, 100, 50_000
coarseness = 1000

def increment(nloop, lock, coarsness):
    assert nloop % coarseness == 0, "Cannot evenly divide work"
    global counter
    for chunk in range(nloop//coarseness):
        with lock:
            for _ in range(coarseness):
                counter += 1

threads, now = [], time()
for _ in range(nthread):
    t = Thread(target=increment, args=(nloop, lock, coarseness))
    threads.append(t)
    t.start()

In [32]:
while alive := sum(t.is_alive() for t in threads):
    sleep(5)
    print("Num threads alive:", sum(t.is_alive() for t in threads))
    
print(f"{nloop:,} loops X {nthread:,} threads -> counter is {counter:,}")
print(f"Running everything took {time()-now:.2f} seconds!")

50,000 loops X 100 threads -> counter is 5,000,000
Running everything took 6.11 seconds!


## Summary:

We've seen the importance of keeping our critical sections safe, to avoid race conditions. But there's no free lunch. To prevent race conditions we have to use synchronization mechanisms, and as we saw, that can carry other issues.

In the next section we'll explore one of the many things that can go wrong with manual synchronization: one of the the scariest words in computer science: **Deadlocks**.